# suscriptor

In [16]:
import ssl
import sys
import psycopg2
import paho.mqtt.client
import json
from sqlalchemy import create_engine
import pandas as pd
import sub 
import datetime 

# conexión con la base de datos (ventas)

In [2]:
host='localhost'
user ='postgres'
password='crush7878'
dbname='ventas'

In [3]:
connection_db_ventas = psycopg2.connect(host = host,
                                user= user, password =password,
                                dbname= dbname)

# creación de tablas

In [4]:
%%time
query_file = open("Data/ventas.sql",'r') 
query = query_file.read()
cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

Wall time: 38 ms


# borrar data

In [5]:
%%time
query_file = open("Data/delete-data-ventas.sql",'r') 
query = query_file.read()
cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

Wall time: 87.9 ms


# agregar data 

In [6]:
query_file = open("Data/data-ventas.sql",'r', encoding='utf-8') 
query = query_file.read()
cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

# conexión con la base de datos (inventario)

In [7]:
host='localhost'
user ='postgres'
password='crush7878'
dbname='inventario'

In [8]:
connection_db_inventario = psycopg2.connect(host = host,
                                user= user, password =password,
                                dbname= dbname)

# creación de tablas

In [9]:
%%time
query_file = open("Data/inventario.sql",'r') 
query = query_file.read()
cur = connection_db_inventario.cursor()
cur.execute(query)
cur.close()
connection_db_inventario.commit()

Wall time: 39 ms


# borrar data

In [10]:
%%time
query_file = open("Data/delete-data-inventario.sql",'r') 
query = query_file.read()
cur = connection_db_inventario.cursor()
cur.execute(query)
cur.close()
connection_db_inventario.commit()

Wall time: 359 ms


# agregar data

In [11]:
query_file = open("Data/data-inventario.sql",'r', encoding='utf-8') 
query = query_file.read()
cur = connection_db_inventario.cursor()
cur.execute(query)
cur.close()
connection_db_inventario.commit()

# conexión con la base de datos (esquema estrella ventas)

In [17]:
host='echo.db.elephantsql.com'
user ='fvrwybpi'
password='Vm7b24osfoJddQLWdc6VotSmYBpOn0UT'
dbname='fvrwybpi'

In [18]:
connection_star_schema_ventas = psycopg2.connect(host = host,
                                user= user, password =password,
                                dbname= dbname)

# creación de tablas

In [19]:
%%time
query_file = open("Data/star_schema_ventas.sql",'r') 
query = query_file.read()
cur = connection_star_schema_ventas.cursor()
cur.execute(query)
cur.close()
connection_star_schema_ventas.commit()

Wall time: 2.57 s


# conexión con la base de datos (esquema estrella inventario)

In [13]:
host='echo.db.elephantsql.com'
user ='nvcmxuqc'
password='Zb2xa6xreimFVSFt5uXM_F_TBCQBzy3l'
dbname='nvcmxuqc'

In [14]:
connection_star_schema_inventario = psycopg2.connect(host = host,
                                user= user, password =password,
                                dbname= dbname)

# creación de tablas

In [15]:
%%time
query_file = open("Data/star_schema_inventario.sql",'r') 
query = query_file.read()
cur = connection_star_schema_inventario.cursor()
cur.execute(query)
cur.close()
connection_star_schema_inventario.commit()

Wall time: 3.59 s


# pretty select

In [24]:
def pretty_select(query):
    db_connection = 'postgres://{}:{}@{}:5432/{}'.format(user, password, host, dbname)
    try:
        engine = create_engine(db_connection)
        dataframe = pd.read_sql_query(query, engine)
    except Exception as e:
        print('Error en el query:', e)
    else:
        engine.dispose()
        return dataframe

# queries

## 1.

Cuales son los clientes conocidos y desconocidos que mas han comprado en cada una de las tiendas.
Este query trae los clientes que mas han comprado por sucursal y que estan dentro del programa de fidelidad y cuenta cada vez que compran por sucursal, se agrupan por sucusal se ordenan por cantidad de veces que compran. Luego del union vienen los clientes que no estan en el plan de fidelidad.

In [51]:
query = """ 
select count(cliente.id_cliente), cliente.id_cliente, cliente.nombre, sucursal.ubicacion from cliente 
join factura on cliente.id_cliente = factura.id_cliente join sucursal on factura.id_sucursal = sucursal.id_sucursal 
where cliente.id_cliente in (select distinct id_cliente from cuenta_programa_fidelidad) group by 2,3,4 order by 1 ASC limit 3"""
pretty_select(query)

,count,id_cliente,nombre,ubicacion
0,1,34,Miguelita,Terrazas del Ávila
1,1,17,Alanna,Terrazas del Ávila
2,1,25,Wilt,Terrazas del Ávila


In [52]:
query = """
select count(cliente.id_cliente), cliente.id_cliente, cliente.nombre, sucursal.ubicacion from cliente 
join factura on cliente.id_cliente = factura.id_cliente join sucursal on factura.id_sucursal = sucursal.id_sucursal 
where cliente.id_cliente not in (select distinct id_cliente from cuenta_programa_fidelidad) group by 2,3,4 order by 1 ASC limit 3"""
pretty_select(query)

,count,id_cliente,nombre,ubicacion
0,1,18,Ashlan,Los Palos Grandes
1,1,27,Brietta,Terrazas del Ávila


## 2.

Diga cual es la categoría de producto que menos se ha vendido en cada una de las tiendas.
Este query cuenta la cantidad de veces que un id_producto sale en un detalle factura, es agrupado por categoria, ordenado de menor a mayor y tiene un limite de 1. Para obtener la categoria que menos sale.

In [53]:
query = """
select count(producto.categoria), producto.categoria, factura.id_sucursal from producto 
join detalle_factura on producto.id_producto = detalle_factura.id_producto join factura 
on detalle_factura.id_factura = factura.id_factura where factura.id_sucursal = 1 group by 2,3 order by 1 ASC limit 1
"""
pretty_select(query)

,count,categoria,id_sucursal
0,1,Cereales,1


In [54]:
query = """
select count(producto.categoria), producto.categoria, factura.id_sucursal from producto 
join detalle_factura on producto.id_producto = detalle_factura.id_producto join factura 
on detalle_factura.id_factura = factura.id_factura where factura.id_sucursal = 2 group by 2,3 order by 1 ASC limit 1
"""
pretty_select(query)

,count,categoria,id_sucursal
0,1,Frescos,2


## 3.

Cuales es el top 5 de productos que mas se han vendido en cada una de las tiendas. Este query trae la cantidad de productos vendidos, el id del producto, el nombre, y la ubicacion. Los agrupa por id,nombre y ubicacion y se ordena de mayor a menor por la cantidad de producto y se limit a 5 del top 5. 

In [55]:
query = """
select count(producto.id_producto), producto.id_producto, producto.nombre, factura.id_sucursal from producto 
join detalle_factura on producto.id_producto = detalle_factura.id_producto join factura 
on detalle_factura.id_factura = factura.id_factura where factura.id_sucursal = 1 group by 2,3,4 order by 1 DESC limit 5
"""
pretty_select(query)

,count,id_producto,nombre,id_sucursal
0,2,8,Pan de Trigo,1
1,1,3,Mantequilla,1
2,1,7,Crema de Leche,1
3,1,16,Lucky Charms,1
4,1,22,Salami,1


In [56]:
query = """
select count(producto.id_producto), producto.id_producto, producto.nombre, factura.id_sucursal from producto 
join detalle_factura on producto.id_producto = detalle_factura.id_producto join factura 
on detalle_factura.id_factura = factura.id_factura where factura.id_sucursal = 2 group by 2,3,4 order by 1 DESC limit 5
"""
pretty_select(query)

,count,id_producto,nombre,id_sucursal
0,2,35,Leche en Polvo,2
1,1,32,Harina,2
2,1,44,Limón,2
3,1,45,Huevos,2


## 4.

Diga todos los clientes que en los últimos 7 días a partir de la ejecución del query han comprado solo en una tienda y los que han comprado en las dos tiendas (realice queries distintos).

In [57]:
query ="""
select cliente.id_cliente, cliente.nombre, cliente.apellido from cliente 
where exists (select distinct id_cliente from factura where factura.id_cliente = cliente.id_cliente and 
factura.fecha_hora > current_date - interval '7 days' group by 1 having count(distinct id_sucursal) = 1)
"""
pretty_select(query)

,id_cliente,nombre,apellido
0,27,Brietta,Peers
1,17,Alanna,Biddlecombe
2,18,Ashlan,Rhymes
3,25,Wilt,Chrishop
4,29,Jacob,Belleny
5,34,Miguelita,Fonquernie


In [58]:
query ="""
select cliente.id_cliente, cliente.nombre, cliente.apellido from cliente 
where exists (select distinct id_cliente from factura where factura.id_cliente = cliente.id_cliente 
and factura.fecha_hora > current_date - interval '7 days' group by 1 having count(distinct id_sucursal) > 1)
"""
pretty_select(query)

,id_cliente,nombre,apellido
0,40,Kacey,Oehm


## 5.

Haga un análisis por estante de cuales categorías de producto tienen mayor rotación de inventario y haga una propuesta de modificación

In [59]:
query ="""
SELECT COUNT(I.id_producto), P.categoria, A.id_estante FROM inventario AS I INNER JOIN producto AS P ON I.id_producto = P.id_producto 
INNER JOIN producto_asignado AS A ON I.id_producto = A.id_producto 
GROUP BY A.id_estante, P.categoria ORDER BY COUNT(I.id_producto) DESC;
"""
pretty_select(query)

,count,categoria,id_estante
0,6,No Perecederos,12
1,5,Vegetales,13
2,5,No Perecederos,11
3,4,Lácteos,1
4,4,Frutas,14
5,4,Cereales,6
6,4,Panadería,3
7,4,Frutas,15
8,3,Enlatados,9
9,3,Bebidas,4


## 6.

Encuentre a todos los clientes que han pagado con 2 bancos distintos en la ultima semana. Enumérelos y diga si son parte del programa de afiliados.

In [60]:
query ="""
select cliente.id_cliente, cliente.nombre, cliente.apellido from cliente 
where exists (select distinct id_cliente from factura where factura.id_cliente = cliente.id_cliente 
and factura.fecha_hora > current_date - interval '7 days' group by 1 having count(distinct banco) > 1)
"""
pretty_select(query)

,id_cliente,nombre,apellido
0,40,Kacey,Oehm
1,29,Jacob,Belleny


# procedimientos almacenados 

## 1.

Realizar compra: Esto procedimiento almacenado debe tener como parámetro una lista de productos con su cantidad, el banco por donde se va a cancelar y un cliente. Debe tener toda la lógica para ingresar en las distintas tablas la compra realizada.

In [27]:
query="""create or replace function realizar_pago(n_cuenta integer, id_cliente integer, banco varchar(10), 
id_sucursal integer, id_factura integer, total numeric(8,0), descuento numeric(8,0), total_pago numeric(8,0), fecha_hora timestamp) returns VOID AS
    $$
    BEGIN
        INSERT into factura (id_factura, id_cliente, fecha_hora, banco, id_sucursal, total, descuento, total_pago) VALUES
        (id_factura, id_cliente, fecha_hora, banco, id_sucursal, total, descuento, total_pago);
    END;
    $$
language plpgsql;"""


cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

## 2.

Haga un procedimiento almacenado que al final del día saque un estado de cuenta de cuanto dinero se tiene en cada banco.

In [28]:
query="""create or replace function estado_cuenta () returns table(banco varchar(10),total_pago numeric) AS
    $$
    BEGIN
    return QUERY 
    (SELECT factura.banco, SUM(factura.total_pago) FROM factura GROUP BY factura.banco); 
    END;
    $$
language plpgsql;"""


cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

In [61]:
query="""select estado_cuenta()"""
pretty_select(query)

,estado_cuenta
0,"(Mercantil,576)"
1,"(Banesco,1904)"
2,"(Provincial,1219)"


## 3.

Haga un procedimiento almacenado que al final del mes saque un estado de cuenta de los puntos de todos los afiliados al programa de fidelidad que compraron ese mes.

In [29]:
query="""create or replace function estado_afiliados () returns table(no_cuenta integer,
          id_cliente integer,puntos integer) AS
    $$
    BEGIN
    return QUERY
    SELECT cuenta_programa_fidelidad.no_cuenta,cuenta_programa_fidelidad.id_cliente,
    cuenta_programa_fidelidad.puntos FROM cuenta_programa_fidelidad,factura 
    WHERE (NOW()- INTERVAL '30' DAY) <= fecha_hora::Timestamp::Date; 
    END;
    $$
language plpgsql;"""


cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

In [30]:
query="""select estado_afiliados()"""
pretty_select(query)

NameError: name 'pretty_select' is not defined

# extras

## 4.

Procedimiento almacenado que retorna True si un cliente pertenece al programa de fidelidad.

In [31]:
query="""
CREATE OR REPLACE FUNCTION verificar_pertenece_programa(id integer) RETURNS BOOLEAN AS
$$
DECLARE pertenece BOOLEAN;
BEGIN
    IF id = (SELECT id_cliente FROM cuenta_programa_fidelidad WHERE id_cliente = id) 
    THEN pertenece = True;
    ELSE
        pertenece = False;
    END IF;
    
    RETURN pertenece;
END
$$
  LANGUAGE 'plpgsql';
"""
cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

## 5.

Procedimiento almacenado que retorna True si la cantidad restante de productos en un estante es menor al 20% de su capacidad.

In [12]:
query="""
CREATE OR REPLACE FUNCTION verificar_cantidad(id integer) RETURNS BOOLEAN AS
$$
DECLARE recargar_estante BOOLEAN;
BEGIN
    SELECT (SUM(cantidad_restante) < 
    (SELECT capacidad*0.20 FROM estante_inteligente WHERE id_estante = id))
    INTO recargar_estante FROM producto_asignado WHERE id_estante = id;
    
    RETURN recargar_estante;
END
$$
  LANGUAGE 'plpgsql';
"""
cur = connection_db_inventario.cursor()
cur.execute(query)
cur.close()
connection_db_inventario.commit()

## 6.

Procedimiento almacenado que llena un estante.

In [13]:
query="""
CREATE OR REPLACE FUNCTION llenar_estante(id integer, fecha_hora timestamp) RETURNS VOID AS
$$
BEGIN
    WITH cantidades_iniciales(id_p, cantidad_i, cantidad_r) AS (SELECT id_producto, cantidad_inicial, cantidad_restante FROM producto_asignado 
    WHERE id_estante = id)
    INSERT INTO producto_asignado(id_producto, fecha_hora, id_estante, cantidad_inicial, cantidad_restante) 
    VALUES (
    (SELECT C.id_p FROM cantidades_iniciales AS C WHERE id_producto = C.id_p),
    fecha_hora, 
    id,
    (SELECT C.cantidad_i FROM cantidades_iniciales AS C WHERE id_producto = C.id_p),
    (SELECT C.cantidad_i FROM cantidades_iniciales AS C WHERE id_producto = C.id_p));
END
$$
  LANGUAGE 'plpgsql';
"""
cur = connection_db_inventario.cursor()
cur.execute(query)
cur.close()
connection_db_inventario.commit()

## 7.

Procedimiento almacenado que actualiza la cantidad restante de un producto de un estante. 

In [14]:
query="""
CREATE OR REPLACE FUNCTION update_estante(id_p integer, cant integer, fecha_hora timestamp) RETURNS VOID AS
$$
BEGIN
    INSERT INTO producto_asignado(id_producto, fecha_hora, id_estante, cantidad_inicial, cantidad_restante) 
    VALUES (id_p, fecha_hora, (SELECT id_estante FROM producto_asignado WHERE id_producto = id_p LIMIT 1), 
    (SELECT cantidad_inicial FROM producto_asignado WHERE id_producto = id_p LIMIT 1), 
    (SELECT cantidad_inicial-cant FROM producto_asignado WHERE id_producto = id_p LIMIT 1)); 
END
$$
  LANGUAGE 'plpgsql';
"""
cur = connection_db_inventario.cursor()
cur.execute(query)
cur.close()
connection_db_inventario.commit()

In [35]:
query="""
CREATE OR REPLACE FUNCTION verificar_cliente(id integer) RETURNS BOOLEAN AS
$$
DECLARE is_cliente BOOLEAN;
BEGIN
    IF 
        id IN (SELECT id_cliente FROM cliente)
        THEN is_cliente = True;
    ELSE
        is_cliente = False;
    
    END IF;
    
    RETURN is_cliente;
END
$$
  LANGUAGE 'plpgsql';
"""
cur = connection_db_ventas.cursor()
cur.execute(query)
connection_db_ventas.commit()

## 8.

In [36]:
query="""
CREATE OR REPLACE FUNCTION verificar_cola(id integer) RETURNS BOOLEAN AS
$$
DECLARE esta_en_cola BOOLEAN;
BEGIN
    IF 
        id IN (SELECT id_cliente FROM cliente_charcuteria)
        THEN esta_en_cola = True;
    ELSE
        esta_en_cola = False;
    
    END IF;
    
    RETURN esta_en_cola;
END
$$
  LANGUAGE 'plpgsql';
"""
cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

# triggers

## 1.

Cada vez que entre un cliente conocido, se debe agregar 1 punto a su estado de cuenta del programa de fidelidad.

In [37]:
query="""
CREATE OR REPLACE FUNCTION add_punto_programa() RETURNS TRIGGER AS
$$
BEGIN
    IF NEW.id_cliente IN (SELECT id_cliente FROM visita) THEN 
    UPDATE cuenta_programa_fidelidad SET puntos = (SELECT puntos+1 FROM cuenta_programa_fidelidad WHERE id_cliente = NEW.id_cliente) 
    WHERE id_cliente = NEW.id_cliente;
    END IF;
    
    RETURN NEW;
END;
$$
  LANGUAGE 'plpgsql';
"""
cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

In [38]:
query="""
CREATE TRIGGER add_punto_programa
  AFTER INSERT 
  ON factura
  FOR EACH ROW
  EXECUTE PROCEDURE add_punto_programa();
"""
cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

In [43]:
query= """
select * from cuenta_programa_fidelidad;
"""
pretty_select(query)

,no_cuenta,id_cliente,fecha_suscripcion,fecha_finalizacion,foto,puntos
0,0,0,0001-01-01,0001-01-01,Ninguna,0
1,1,20,2020-07-01,2020-07-31,http://dummyimage.com/109x112.jpg/ff4444/ffffff,0
2,2,47,2020-07-01,2020-07-31,http://dummyimage.com/199x158.jpg/ff4444/ffffff,0
3,6,36,2020-07-01,2020-07-31,http://dummyimage.com/203x129.jpg/ff4444/ffffff,0
4,8,33,2020-07-01,2020-07-31,http://dummyimage.com/249x205.png/dddddd/000000,0
5,9,48,2020-07-01,2020-07-31,http://dummyimage.com/228x238.png/cc0000/ffffff,0
6,13,50,2020-07-01,2020-07-31,http://dummyimage.com/145x226.jpg/ff4444/ffffff,0
7,14,24,2020-07-01,2020-07-31,http://dummyimage.com/117x161.png/cc0000/ffffff,0
8,15,12,2020-07-01,2020-07-31,http://dummyimage.com/156x122.png/cc0000/ffffff,0
9,16,38,2020-07-01,2020-07-31,http://dummyimage.com/164x124.jpg/ff4444/ffffff,0


## 2.

Cada vez que un cliente conocido realice una compra, se le agregaran los puntos en su programa de fidelidad.

In [39]:
query="""
CREATE OR REPLACE FUNCTION add_puntos() RETURNS TRIGGER AS
$$
BEGIN
    IF (SELECT verificar_pertenece_programa(NEW.id_cliente)) = True 
    THEN UPDATE cuenta_programa_fidelidad SET puntos = (SELECT puntos+(NEW.total_pago*0.1) FROM cuenta_programa_fidelidad 
    WHERE no_cuenta = (SELECT no_cuenta FROM cuenta_programa_fidelidad WHERE id_cliente = NEW.id_cliente)) 
    WHERE no_cuenta = (SELECT no_cuenta FROM cuenta_programa_fidelidad WHERE id_cliente = NEW.id_cliente);
    END IF;
    
    RETURN NEW;
END;
$$
  LANGUAGE 'plpgsql';
"""
cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

In [40]:
query="""
CREATE TRIGGER add_puntos
  AFTER INSERT 
  ON factura
  FOR EACH ROW
  EXECUTE PROCEDURE add_puntos();
"""
cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

In [38]:
query= """
select * from cliente;
"""
pretty_select(query)

,id_cliente,nombre,apellido,telefono,direccion
0,40,Kacey,Oehm,1.252790e+09,51 Ludington Center
1,3,Gareth,Moxson,2.742826e+09,74 Lakewood Gardens Hill
2,15,Craggy,Bottell,4.789100e+09,67359 Sherman Center
3,10,Benson,Thoumas,8.983431e+09,74 North Road
4,26,Chariot,Karpinski,2.878758e+09,00 Village Center
5,27,Brietta,Peers,3.542742e+09,066 Kings Road
6,0,Desconocido,Desconocido,0.000000e+00,Desconocida
7,1,Vania,Tamplin,8.733382e+09,3 Grover Pass
8,4,Nollie,Bertram,7.782379e+09,5012 Northview Plaza
9,5,Tann,Steabler,8.346899e+09,121 Bultman Pass


In [39]:
query= """
select * from factura;
"""
pretty_select(query)

,id_factura,id_cliente,fecha_hora,banco,id_sucursal,total,descuento,total_pago
0,1,29,2020-07-01 18:00:00.122570,Provincial,2,384.0,0.0,384.0
1,2,18,2020-07-01 18:00:00.705029,Banesco,2,1904.0,0.0,1904.0
2,3,27,2020-07-01 18:00:00.101037,Mercantil,1,2.0,0.0,2.0
3,4,17,2020-07-01 18:00:00.704098,Provincial,1,325.0,0.0,325.0
4,5,40,2020-07-01 18:00:00.031355,Provincial,2,390.0,0.0,390.0
5,6,29,2020-07-01 18:00:00.031355,Mercantil,2,438.0,0.0,438.0
6,7,25,2020-07-01 18:00:00.066330,Provincial,1,120.0,0.0,120.0
7,8,34,2020-07-01 18:00:00.531155,Mercantil,1,95.0,0.0,95.0
8,9,40,2020-07-01 23:04:33.557845,Mercantil,1,41.0,0.0,41.0


In [40]:
query= """
select * from cuenta_programa_fidelidad;
"""
pretty_select(query)

,no_cuenta,id_cliente,fecha_suscripcion,fecha_finalizacion,foto,puntos
0,0,0,0001-01-01,0001-01-01,Ninguna,0
1,1,20,2020-07-01,2020-07-31,http://dummyimage.com/109x112.jpg/ff4444/ffffff,0
2,2,47,2020-07-01,2020-07-31,http://dummyimage.com/199x158.jpg/ff4444/ffffff,0
3,6,36,2020-07-01,2020-07-31,http://dummyimage.com/203x129.jpg/ff4444/ffffff,0
4,8,33,2020-07-01,2020-07-31,http://dummyimage.com/249x205.png/dddddd/000000,0
5,9,48,2020-07-01,2020-07-31,http://dummyimage.com/228x238.png/cc0000/ffffff,0
6,13,50,2020-07-01,2020-07-31,http://dummyimage.com/145x226.jpg/ff4444/ffffff,0
7,14,24,2020-07-01,2020-07-31,http://dummyimage.com/117x161.png/cc0000/ffffff,0
8,15,12,2020-07-01,2020-07-31,http://dummyimage.com/156x122.png/cc0000/ffffff,0
9,16,38,2020-07-01,2020-07-31,http://dummyimage.com/164x124.jpg/ff4444/ffffff,0


## 3.

Si un cliente desconocido con la misma cedula compra mas de 4 veces, automáticamente se agregará al programa de fidelidad.

In [41]:
query="""
CREATE OR REPLACE FUNCTION add_to_programa() RETURNS TRIGGER AS
$$
BEGIN
    IF (SELECT COUNT(id_cliente) > 4 FROM factura) THEN 
    INSERT INTO cuenta_programa_fidelidad (no_cuenta, id_cliente, fecha_suscripcion, fecha_finalizacion) VALUES 
    ((SELECT no_cuenta+1 FROM cuenta_programa_fidelidad ORDER BY no_cuenta DESC LIMIT 1), NEW.id_cliente, DATE(NEW.fecha_hora), 
    DATE(NEW.fecha_hora)+30);
    END IF;
    
    RETURN NEW;
END;
$$
  LANGUAGE 'plpgsql';
"""
cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

In [42]:
query="""
CREATE TRIGGER add_to_programa
  AFTER INSERT 
  ON factura
  FOR EACH ROW
  EXECUTE PROCEDURE add_to_programa();
"""
cur = connection_db_ventas.cursor()
cur.execute(query)
cur.close()
connection_db_ventas.commit()

In [41]:
query= """
select * from factura;
"""
pretty_select(query)

,id_factura,id_cliente,fecha_hora,banco,id_sucursal,total,descuento,total_pago
0,1,29,2020-07-01 18:00:00.122570,Provincial,2,384.0,0.0,384.0
1,2,18,2020-07-01 18:00:00.705029,Banesco,2,1904.0,0.0,1904.0
2,3,27,2020-07-01 18:00:00.101037,Mercantil,1,2.0,0.0,2.0
3,4,17,2020-07-01 18:00:00.704098,Provincial,1,325.0,0.0,325.0
4,5,40,2020-07-01 18:00:00.031355,Provincial,2,390.0,0.0,390.0
5,6,29,2020-07-01 18:00:00.031355,Mercantil,2,438.0,0.0,438.0
6,7,25,2020-07-01 18:00:00.066330,Provincial,1,120.0,0.0,120.0
7,8,34,2020-07-01 18:00:00.531155,Mercantil,1,95.0,0.0,95.0
8,9,40,2020-07-01 23:04:33.557845,Mercantil,1,41.0,0.0,41.0


In [42]:
query= """
select * from cuenta_programa_fidelidad;
"""
pretty_select(query)

,no_cuenta,id_cliente,fecha_suscripcion,fecha_finalizacion,foto,puntos
0,0,0,0001-01-01,0001-01-01,Ninguna,0
1,1,20,2020-07-01,2020-07-31,http://dummyimage.com/109x112.jpg/ff4444/ffffff,0
2,2,47,2020-07-01,2020-07-31,http://dummyimage.com/199x158.jpg/ff4444/ffffff,0
3,6,36,2020-07-01,2020-07-31,http://dummyimage.com/203x129.jpg/ff4444/ffffff,0
4,8,33,2020-07-01,2020-07-31,http://dummyimage.com/249x205.png/dddddd/000000,0
5,9,48,2020-07-01,2020-07-31,http://dummyimage.com/228x238.png/cc0000/ffffff,0
6,13,50,2020-07-01,2020-07-31,http://dummyimage.com/145x226.jpg/ff4444/ffffff,0
7,14,24,2020-07-01,2020-07-31,http://dummyimage.com/117x161.png/cc0000/ffffff,0
8,15,12,2020-07-01,2020-07-31,http://dummyimage.com/156x122.png/cc0000/ffffff,0
9,16,38,2020-07-01,2020-07-31,http://dummyimage.com/164x124.jpg/ff4444/ffffff,0


# extras

## 4.

Si la cantidad de producto en el inventario es suficiente para llenar el estante se procede a realizar el update, si no lanza un excepción.

# preguntas

## 1.

¿Cuáles son las categorías de producto que se venden mejor en cada una de las tiendas? Use un pivot table para razonar su respuesta.

In [63]:
query="""SELECT * FROM crosstab($$
select detalle_factura.id_producto, producto.categoria,  factura.id_sucursal from factura join detalle_factura on
factura.id_factura = detalle_factura.id_factura join producto on detalle_factura.id_producto = producto.id_producto where factura.id_sucursal = 1 $$, 
$$ SELECT 'Lácteos' UNION ALL
  SELECT 'Charcutería' UNION ALL
  SELECT 'Panadería' UNION ALL
  SELECT 'Bebidas' UNION ALL
  SELECT 'Cereales' UNION ALL
  SELECT 'Galletas' UNION ALL
  SELECT 'No Perecederos' UNION ALL
SELECT 'Vegetales' UNION ALL 
SELECT 'Frutas' UNION ALL
 SELECT 'Frescos' UNION ALL  
SELECT 'Enlatados' $$
) AS ct(categoria text, Lácteos text, Charcutería text, Panadería text, Bebidas text, Cereales text,
Galletas text, No_Perecederos text, Vegetales text, Frutas text, Frescos text, Enlatados text);"""
pretty_select(query)

,categoria,lácteos,charcutería,panadería,bebidas,cereales,galletas,no_perecederos,vegetales,frutas,frescos,enlatados
0,16,None,None,None,None,1,None,None,None,None,None,None
1,8,None,None,1,None,None,None,None,None,None,None,None
2,22,None,1,None,None,None,None,None,None,None,None,None
3,8,None,None,1,None,None,None,None,None,None,None,None
4,7,1,None,None,None,None,None,None,None,None,None,None
5,3,1,None,None,None,None,None,None,None,None,None,None


In [64]:
query="""SELECT * FROM crosstab($$
select detalle_factura.id_producto, producto.categoria,  factura.id_sucursal from factura join detalle_factura on
factura.id_factura = detalle_factura.id_factura join producto on detalle_factura.id_producto = producto.id_producto where factura.id_sucursal = 2 $$, 
$$ SELECT 'Lácteos' UNION ALL
  SELECT 'Charcutería' UNION ALL
  SELECT 'Panadería' UNION ALL
  SELECT 'Bebidas' UNION ALL
  SELECT 'Cereales' UNION ALL
  SELECT 'Galletas' UNION ALL
  SELECT 'No Perecederos' UNION ALL
SELECT 'Vegetales' UNION ALL 
SELECT 'Frutas' UNION ALL
 SELECT 'Frescos' UNION ALL  
SELECT 'Enlatados' $$
) AS ct(categoria text, Lácteos text, Charcutería text, Panadería text, Bebidas text, Cereales text,
Galletas text, No_Perecederos text, Vegetales text, Frutas text, Frescos text, Enlatados text);"""
pretty_select(query)

,categoria,lácteos,charcutería,panadería,bebidas,cereales,galletas,no_perecederos,vegetales,frutas,frescos,enlatados
0,35,None,None,None,None,None,None,2,None,None,None,None
1,45,None,None,None,None,None,None,None,None,None,2,None
2,44,None,None,None,None,None,None,None,None,2,None,None
3,35,None,None,None,None,None,None,2,None,None,None,None
4,32,None,None,None,None,None,None,2,None,None,None,None


## 2.

¿Qué banco, categoría de producto y tienda prefieren los clientes de nuestro programa de afiliados? Razone su respuesta.

In [65]:
query= """
select count(factura.id_factura), factura.banco from factura join cuenta_programa_fidelidad on 
factura.id_cliente = cuenta_programa_fidelidad.id_cliente group by 2;
"""
pretty_select(query)

,count,banco
0,4,Mercantil
1,5,Provincial


In [66]:
query= """
select count(factura.id_factura), factura.id_sucursal from factura join cuenta_programa_fidelidad on 
factura.id_cliente = cuenta_programa_fidelidad.id_cliente group by 2;
"""
pretty_select(query)

,count,id_sucursal
0,5,1
1,4,2


In [67]:
query= """
select count(detalle_factura.id_producto), producto.categoria from factura join detalle_factura on factura.id_factura = detalle_factura.id_factura 
join cuenta_programa_fidelidad on factura.id_cliente = cuenta_programa_fidelidad.id_cliente 
join producto on producto.id_producto = detalle_factura.id_producto group by 2;
"""
pretty_select(query)

,count,categoria
0,3,Lácteos
1,2,Panadería
2,3,No Perecederos
3,1,Charcutería
4,2,Frutas


## 3.

¿Qué horas del día son más rentable para cada una de las tiendas? Razone su respuesta.

In [49]:
query="""SELECT COUNT(id_cliente), fecha_hora FROM factura WHERE id_sucursal = 1 GROUP BY id_cliente,fecha_hora """
pretty_select(query)

,count,fecha_hora
0,1,2020-07-01 18:00:00.704098
1,1,2020-07-01 18:00:00.066330
2,1,2020-07-01 18:00:00.101037
3,1,2020-07-01 18:00:00.531155
4,1,2020-07-01 23:04:33.557845


In [50]:
query="""SELECT COUNT(id_cliente), fecha_hora FROM factura WHERE id_sucursal = 2 GROUP BY id_cliente,fecha_hora """
pretty_select(query)

,count,fecha_hora
0,1,2020-07-01 18:00:00.705029
1,1,2020-07-01 18:00:00.031355
2,1,2020-07-01 18:00:00.122570
3,1,2020-07-01 18:00:00.031355


# conectar e iniciar suscriptor

In [ ]:
## se crea un suscriptor y se conecta 
suscriptor = sub.suscriptor(connection_db_ventas, connection_db_inventario)
suscriptor.conectar_suscriptor()

connected (b'')
{'id_producto': 31, 'cantidad': 7, 'id_estante': 11, 'sucursal': 'sucursal2', 'hora': '2020-07-19 11:14:53.237461'}
---------------------------------------------------------
datos guardados
{'id_producto': 38, 'cantidad': 5, 'id_estante': 13, 'sucursal': 'sucursal2', 'hora': '2020-07-19 11:15:07.784751'}
---------------------------------------------------------
datos guardados
{'sucursal': 'sucursal1', 'id_cliente': 863, 'id_sucursal': 1}
---------------------------------------------------------
{'id_producto': 28, 'cantidad': 1, 'id_estante': 7, 'sucursal': 'sucursal1', 'hora': '2020-07-19 11:15:29.870585'}
---------------------------------------------------------
datos guardados
{'id_producto': 45, 'cantidad': 6, 'id_estante': 16, 'sucursal': 'sucursal2', 'hora': '2020-07-19 11:15:43.957363'}
---------------------------------------------------------
datos guardados
{'id_producto': 39, 'cantidad': 6, 'id_estante': 14, 'sucursal': 'sucursal2', 'hora': '2020-07-19 11:15:

In [2]:
print(datetime.datetime.now())

2020-07-17 10:12:28.477175
